<a href="https://colab.research.google.com/github/Mogarthron/Magazynier/blob/master/PIANKI_GENERATOR_BAZY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls drive/MyDrive/ | grep "PIAGEN"

PIAGEN_wersja_3.82.xlsm
PIAGEN_wersja_3.83.xlsm


In [ ]:
pg_path = "drive/MyDrive/PIAGEN_wersja_3.83.xlsm"

In [ ]:
# # USUNIECIE BAZY PIANEK
#from sqlite3 import connect
#cnn = connect("drive/MyDrive/GENERATORY.db")
# #cnn.execute("DELETE FROM baza_PIANKI")
#cnn.execute("DROP TABLE lista_bryl_pianki")
#cnn.commit()
#cnn.close()

# from sqlite3 import connect
# cnn = connect("drive/MyDrive/GENERATORY.db")
# cnn.execute("DELETE FROM baza_PIANKI")
# cnn.execute("DELETE FROM lista_bryl_pianki")
# cnn.commit()
# cnn.close()


#Data Cleaning and procesing

In [ ]:
import openpyxl
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
pg = openpyxl.load_workbook(pg_path, data_only=True)

In [ ]:
lista_modeli_pianki = dict()
serie_pianek = pg["SERIE"]
for r in serie_pianek:
  if r[0].value != None:
    lis = list()
    for i in range(r[3].value):
      lis.append(str(r[4+i].value))

    lista_modeli_pianki[r[0].value] = "_".join(lis).replace(".", ",")
    # print(r[0].value, r[3].value, "_".join(lis))

del(lista_modeli_pianki["LAGO"])
# lista_modeli_pianki

In [ ]:
lista_modeli_pianki["AMALFI"] = lista_modeli_pianki["AMALFI"] + "_1,5 MEM_3,5 MEM_NW MEM_WN MEM_[RS MEM_RS] MEM_[LA MEM_LA] MEM"
lista_modeli_pianki

{'AMALFI': '1,5_2,5_3_4_][_NT_N124_[LA_[RS_NW_P102x82_P102x102_1,5 MEM_3,5 MEM_NW MEM_WN MEM_[RS MEM_RS] MEM_[LA MEM_LA] MEM',
 'AVANT': '[1]_[2]_[2,5]_[3]_[3F]_[3,5]_[3,5F]_[1_[1,5_[2_[2,5_[3_[3F_[3,5_[3,5F_1]_1,5]_2]_2,5]_3]_3F]_3,5]_3,5F]_1_1,5_2_2,5_3_3,5_N_[LM_[L_LM]_L]_Pufa 110x60_Pufa 114x84_Pufa 104x104_Pufa 114x114_Pufa 64x64_Pufa 84x84_Pufa 104x64_Pufa 112x97_Pufa 112x84_Pufa 112x112_1][_1,5][_2][_2,5][_3][_3F][_3,5][_3,5F][_L][_LM][',
 'CALYPSO': '1_1,5_2,5_4_][_D_E80D_[L2H_125N_116N_[LA_[LAD_P80x100_P90x100',
 'COCO': '[1o]_[1M]',
 'DIVA': 'X1X_X2,5X_X3X_X3,5X_X4X_X5X_2,5X_3X_3,5X_4X_5X_5N_2,5_3_N_LA_[_]_P95x113_P95x95',
 'DUO': '1_1,5_2_2,5_3_3F_3,5_3,5F_T1,5_N_L][_LH][_Pufa 108x108_Pufa 102x102_Pufa 102x82_Pufa 82x82_Pufa 62x62_][',
 'ELIXIR': '2,5_3_3,5_4_[_N_Pufa 98x120_Pufa 98x100',
 'EXTREME': '[1s_[1,5s_[2s_1s_1,5s_2s__[1,5m__1m_1,5m_2m_[Ls_ONs_P80x80_P80x105_P105x105_ZAG Ns_ZAG N125x105_ZAG Nm',
 'GREY': '[1]',
 'HUDSON': '2,5_3_3,5_4_][_[L_[LH_ON_NW 149x159_WN 159x

In [ ]:
sheet = [x for x in pg.sheetnames if len(x) == 2]
# sheet

In [ ]:
poz = list()

for s in sheet:
  for r in pg[s]:
    # print(r[6].column)
    if r[0].value != None:
      poz.append([x.value for x in r if x.column < 18] + [s])

In [ ]:
df = pd.DataFrame(poz)
df.columns = ["MODEL", "BRYLA", "TYP", "PRZEZ", "OR", "OZN", "PROFIL", "NUMER", "wymiar1", "wymiar2", "ilosc", "obietosc_k", "obietosc_f", "14", "15" , "16" ,"ile_ciec", "18"]


In [ ]:
df[["obietosc_k", "obietosc_f"]] = df[["obietosc_k", "obietosc_f"]].fillna(0).astype(np.int32)
df.drop(axis=0, index=df[df.MODEL == "NAZWA"].index, inplace=True)

In [ ]:
_tab = df[df.MODEL != "NAZWA"]

# _tab[_tab.ilosc.isna()]


In [ ]:
tab = _tab[~_tab.ilosc.isna()]

In [ ]:
# tab

In [ ]:
def pars_bryla(x):
  if type(x) != str:
    y = str(x)
    y = y.replace(".0", "").replace(".", ",")

    return y
  return x

def pars_ilosc(x):
  if type(x) != int:
    y = np.int32(x)

    return y
  return x

def wymiary(wymiar, format):
  if format == None:
    return wymiar
  elif format != None:
    return format

def wys(wym):
  try:
    if type(wym) != None:
      return int(wym.split("x")[0].replace("***", ""))

    return 0
  except:
    return 0


def szer(s):
  try:
    return int(s.lower().split("x")[-2].replace("*** ", ""))
  except:
    return None

def dlug(d):
  try:
    return int(d.lower().split("x")[-1])
  except:
    return None

tab["BRYLA"] = tab["BRYLA"].apply(pars_bryla)
tab["ilosc"].fillna(0, inplace=True)
tab["ilosc"] = tab["ilosc"].apply(pars_ilosc)
tab["WYMIAR"] = tab.apply(lambda x: wymiary(x.wymiar1,x.wymiar2), axis=1)
tab["WYS"] = tab["WYMIAR"].apply(wys)
tab["SZER"] = tab["WYMIAR"].apply(szer)
tab["DLUG"] = tab["WYMIAR"].apply(dlug)

In [ ]:
tab = tab[tab.MODEL.isin(list(lista_modeli_pianki.keys()))]

In [ ]:
tab

,MODEL,BRYLA,TYP,PRZEZ,OR,OZN,PROFIL,NUMER,wymiar1,wymiar2,...,obietosc_f,14,15,16,ile_ciec,18,WYMIAR,WYS,SZER,DLUG
28,DUO,"[3,5]",T-2538,korpus,None,prosta,None,AA0003,10x160x1800,None,...,2880000,None,None,None,1,AA,10x160x1800,10,160,1800
29,DUO,"[3,5F]",T-2538,korpus,None,prosta,None,AA0003,10x160x1800,None,...,2880000,None,None,None,1,AA,10x160x1800,10,160,1800
305,SAMOA,N,T-2538,korpus,None,profil,P-9,AA0014,None,70x100x500,...,3500000,\DOKUMENTACJA\RYSUNKI\A\AA0014.gif,None,None,2,AA,70x100x500,70,100,500
438,AVANT,2,T-2538,korpus,None,profil,P-12,AA0022,None,80x100x600,...,4800000,\DOKUMENTACJA\RYSUNKI\A\AA0022.gif,None,None,2,AA,80x100x600,80,100,600
439,AVANT,"2,5",T-2538,korpus,None,profil,P-12,AA0022,None,80x100x600,...,4800000,\DOKUMENTACJA\RYSUNKI\A\AA0022.gif,None,None,2,AA,80x100x600,80,100,600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20571,MAXWELL,P82x82,G-401,siedzisko,None,format,None,FA0101,None,50x800x800,...,32000000,\DOKUMENTACJA\RYSUNKI\F\FA0101.gif,None,None,1,FA,50x800x800,50,800,800
20572,MAXWELL,P103x82,G-401,siedzisko,None,format,None,FA0205,None,50x800x1000,...,40000000,\DOKUMENTACJA\RYSUNKI\F\FA0205.gif,None,None,1,FA,50x800x1000,50,800,1000
20573,MAXWELL,P103x92,G-401,siedzisko,None,format,None,FA0206,None,50x900x1000,...,45000000,\DOKUMENTACJA\RYSUNKI\F\FA0206.gif,None,None,1,FA,50x900x1000,50,900,1000
20574,COCO,[1M],G-401,siedzisko,None,szablon,P-197,FA0375,None,50x930x931,...,43245000,\DOKUMENTACJA\RYSUNKI\F\FA0205.gif,None,None,1,FA,50x930x931,50,930,931


#ZAPIS DO BAZY

In [ ]:
from sqlalchemy import create_engine, text, Column, String, Integer, Numeric
# from sqlalchemy.ext.declarative import
from sqlalchemy.orm import sessionmaker, declarative_base

engine = create_engine("sqlite:///drive/MyDrive/GENERATORY.db", echo=False)

Base = declarative_base()

class Lista_bryl_pianki(Base):
  __tablename__ = "lista_bryl_pianki"

  model = Column("Model", String(25), primary_key=True)
  lista_bryl = Column("lista_brył", String)

  def __init__(self, model, lista_bryl):
    self.model = model
    self.lista_bryl = lista_bryl

  def __repr__(self):
    return f"{self.model}, {len(self.lista_bryl.split('_'))}"





In [ ]:
# tab[['MODEL', 'BRYLA', 'TYP', 'PRZEZ', 'OR', 'OZN', 'PROFIL', 'NUMER','ilosc','WYMIAR', 'WYS', 'SZER', 'DLUG']].to_sql("baza_PIANKI", con=engine, index=False, if_exists="append")
#4427 2023-11-06
#4428 2024-02-09

4428

In [ ]:
Base.metadata.create_all(bind=engine)

Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
for m in lista_modeli_pianki:
  session.add(Lista_bryl_pianki(m, lista_modeli_pianki[m]))

session.commit()

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: lista_bryl_pianki.Model
[SQL: INSERT INTO lista_bryl_pianki ("Model", "lista_brył") VALUES (?, ?)]
[parameters: [('AMALFI', '1,5_2,5_3_4_][_NT_N124_[LA_[RS_NW_P102x82_P102x102_1,5 MEM_3,5 MEM_NW MEM_WN MEM_[RS MEM_RS] MEM_[LA MEM_LA] MEM'), ('AVANT', '[1]_[2]_[2,5]_[3]_[3F]_[3,5]_[3,5F]_[1_[1,5_[2_[2,5_[3_[3F_[3,5_[3,5F_1]_1,5]_2]_2,5]_3]_3F]_3,5]_3,5F]_1_1,5_2_2,5_3_3,5_N_[LM_[L_LM]_L]_Pufa 110x60 ... (11 characters truncated) ... 4_Pufa 104x104_Pufa 114x114_Pufa 64x64_Pufa 84x84_Pufa 104x64_Pufa 112x97_Pufa 112x84_Pufa 112x112_1][_1,5][_2][_2,5][_3][_3F][_3,5][_3,5F][_L][_LM]['), ('CALYPSO', '1_1,5_2,5_4_][_D_E80D_[L2H_125N_116N_[LA_[LAD_P80x100_P90x100'), ('COCO', '[1o]_[1M]'), ('DIVA', 'X1X_X2,5X_X3X_X3,5X_X4X_X5X_2,5X_3X_3,5X_4X_5X_5N_2,5_3_N_LA_[_]_P95x113_P95x95'), ('DUO', '1_1,5_2_2,5_3_3F_3,5_3,5F_T1,5_N_L][_LH][_Pufa 108x108_Pufa 102x102_Pufa 102x82_Pufa 82x82_Pufa 62x62_]['), ('ELIXIR', '2,5_3_3,5_4_[_N_Pufa 